<a href="https://colab.research.google.com/github/CLAUDERNORONHA/MACHINE/blob/main/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CatBoost





O principal motivo de eu usar CatBoost é que ele é fácil de usar, eficiente e funciona especialmente bem com variáveis ​​categóricas. Como o nome indica, CatBoost significa aumento ' categórico '. É mais rápido de usar do que, digamos, XGBoost, porque não requer o uso de pré-processamento de seus dados, o que pode levar muito tempo em um processo típico de construção de modelo de Ciência de Dados. Outro problema que outros algoritmos têm é, ao usar variáveis ​​categóricas como IDs, eles criam uma matriz impossível de calcular composta de milhares de colunas feitas de variáveis ​​fictícias ou codificação one-hot. CatBoost corrige esse problema na forma como transforma suas variáveis ​​categóricas, como você verá a seguir.





Treinamento

CatBoost se baseia em árvores de decisão com gradiente aumentado, incluindo um conjunto de dados de treinamento, com precisão determinada em um conjunto de dados de validação. No treinamento, essas árvores de decisão são construídas consecutivamente com cada árvore tendo sua perda reduzida.




Cálculo de divisão

Com base nos parâmetros iniciais do CatBoost, a quantização é usada para os recursos numéricos ao determinar as melhores maneiras de dividir os dados em intervalos.

Transformação de recursos categóricos

O principal benefício, eu acho, desse algoritmo é que ele trata a transformação de recursos categóricos da melhor maneira quando comparado a outros algoritmos de aprendizado de máquina.

Por exemplo, na classificação, uma permutação é realizada aleatoriamente, em seguida, um cálculo é realizado a partir de uma fórmula padrão exclusiva para CatBoost ( codificação de destino ordenada ):

CatBoost  - é um algoritmo de machine , ele é um dos algoritmos dos ansambols. Ele é baseado e arvore de decisão.

In [ ]:
#pip install catboost

In [1]:
#Pacotes

import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm


from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
#Criar uma função  para ver as caracteristicas do DAtaframe 

def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    return summary

In [4]:
file_loc = "/content/drive/MyDrive/1-CIENCIA DE DADOS-CURSOS_ESTUDO DE CASO/CATBOOT/bank-full (1).csv"

In [7]:
df = pd.read_csv(file_loc, delimiter=';')

In [8]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [9]:
df.dropna()

#Limpara valores nulos 

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [10]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [15]:
#Vou chamar a função que foi criada 

result = resumetable(df)
result  



Dataset Shape: (45211, 17)


,Name,dtypes,Missing,Uniques,First Value,Second Value
0,age,int64,0,77,58,44
1,job,object,0,12,management,technician
2,marital,object,0,3,married,single
3,education,object,0,4,tertiary,secondary
4,default,object,0,2,no,no
5,balance,int64,0,7168,2143,29
6,housing,object,0,2,yes,yes
7,loan,object,0,2,no,no
8,contact,object,0,3,unknown,unknown
9,day,int64,0,31,5,5


In [17]:
#Separa os dados preditores. 


target_col = 'y' #y vai ser o dado preditor

X = df.loc[:, df.columns != target_col]
y = df.loc[:, target_col]

In [19]:
#Treinar

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33,
                                                    random_state= 42)



In [20]:
#Criar uma lista de features 


cat_features = ["job", "marital", "education", "default", "housing", "loan", "contact","month", "poutcome"]


Criar Maquina Preditiva

In [23]:
#Criar a maquina preditiva 

model_cb = CatBoostClassifier(task_type='CPU', #utilizar o CPU os dados são pequenos, mas podemos utilizar a GPU
                              iterations=100, #100 interações
                              random_state = 2022, #criar uma aleatoriedade no processo 
                              eval_metric="Accuracy",# metrica que a maqui preditiva considere o parendizado 
                              verbose=5)#printar de 5 em  5 

Treinamento 

In [24]:
model_cb.fit(X_train, y_train, cat_features= cat_features, plot=True, eval_set=(X_test, y_test))

#é no treinamneot que que utilizamos o CatBoot

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.200352
0:	learn: 0.8866660	test: 0.8857909	best: 0.8857909 (0)	total: 72.4ms	remaining: 7.17s
5:	learn: 0.8921792	test: 0.8896113	best: 0.8896113 (5)	total: 170ms	remaining: 2.66s
10:	learn: 0.9003004	test: 0.8981233	best: 0.8981233 (10)	total: 265ms	remaining: 2.14s
15:	learn: 0.9040969	test: 0.9010054	best: 0.9022118 (14)	total: 362ms	remaining: 1.9s
20:	learn: 0.9069361	test: 0.9024129	best: 0.9024129 (20)	total: 444ms	remaining: 1.67s
25:	learn: 0.9085537	test: 0.9030831	best: 0.9030831 (25)	total: 538ms	remaining: 1.53s
30:	learn: 0.9097422	test: 0.9045576	best: 0.9045576 (29)	total: 627ms	remaining: 1.39s
35:	learn: 0.9111617	test: 0.9053619	best: 0.9053619 (35)	total: 715ms	remaining: 1.27s
40:	learn: 0.9116899	test: 0.9058981	best: 0.9058981 (40)	total: 805ms	remaining: 1.16s
45:	learn: 0.9126803	test: 0.9067024	best: 0.9069035 (44)	total: 883ms	remaining: 1.04s
50:	learn: 0.9133406	test: 0.9068365	best: 0.9069035 (44)	total: 986ms	remaining: 947ms
55:	le

In [25]:
y_pred  = model_cb.predict(X_test)

In [26]:
accuracy_score(y_test, y_pred)

0.9091823056300268